## Dependencies

In [1]:
from openvaccine_scripts import *
import warnings, json
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers, losses, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


SEED = 0
seed_everything(SEED)
warnings.filterwarnings('ignore')

# Model parameters

In [2]:
config = {
  "BATCH_SIZE": 64,
  "EPOCHS": 120,
  "LEARNING_RATE": 1e-3,
  "ES_PATIENCE": 10,
  "N_FOLDS": 5,
  "N_USED_FOLDS": 5,
  "PB_SEQ_LEN": 107,
  "PV_SEQ_LEN": 130,
}

with open('config.json', 'w') as json_file:
    json.dump(json.loads(json.dumps(config)), json_file)
    
config

{'BATCH_SIZE': 64,
 'EPOCHS': 120,
 'LEARNING_RATE': 0.001,
 'ES_PATIENCE': 10,
 'N_FOLDS': 5,
 'N_USED_FOLDS': 5,
 'PB_SEQ_LEN': 107,
 'PV_SEQ_LEN': 130}

# Load data

In [3]:
database_base_path = '/kaggle/input/stanford-covid-vaccine/'
train = pd.read_json(database_base_path + 'train.json', lines=True)
test = pd.read_json(database_base_path + 'test.json', lines=True)

print('Train samples: %d' % len(train))
display(train.head())
print(f'Test samples: {len(test)}')
display(test.head())

Train samples: 2400


,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


Test samples: 3634


,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


## Auxiliary functions

In [4]:
def get_dataset(x, y=None, labeled=True, shuffled=True, batch_size=32, buffer_size=-1, seed=0):
    if labeled:
        dataset = tf.data.Dataset.from_tensor_slices(({'inputs_seq': x[:, 0, :, :], 
                                                       'inputs_struct': x[:, 1, :, :], 
                                                       'inputs_loop': x[:, 2, :, :], 
                                                       'inputs_bpps_max': x[:, 3, :, :], 
                                                       'inputs_bpps_sum': x[:, 4, :, :], 
                                                       'inputs_bpps_mean': x[:, 5, :, :]}, 
                                                      {'outputs': y}))
    else:
        dataset = tf.data.Dataset.from_tensor_slices(({'inputs_seq': x[:, 0, :, :], 
                                                       'inputs_struct': x[:, 1, :, :], 
                                                       'inputs_loop': x[:, 2, :, :], 
                                                       'inputs_bpps_max': x[:, 3, :, :], 
                                                       'inputs_bpps_sum': x[:, 4, :, :], 
                                                       'inputs_bpps_mean': x[:, 5, :, :]}))
    
    
    if shuffled:
        dataset = dataset.shuffle(2048, seed=seed)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size)
    
    return dataset

def get_dataset_sampling(x, y=None, shuffled=True, seed=0):
    dataset = tf.data.Dataset.from_tensor_slices(({'inputs_seq': x[:, 0, :, :], 
                                                   'inputs_struct': x[:, 1, :, :], 
                                                   'inputs_loop': x[:, 2, :, :], 
                                                   'inputs_bpps_max': x[:, 3, :, :], 
                                                   'inputs_bpps_sum': x[:, 4, :, :], 
                                                   'inputs_bpps_mean': x[:, 5, :, :]}, 
                                                  {'outputs': y}))
    
    
    if shuffled:
        dataset = dataset.shuffle(2048, seed=seed)
    
    return dataset

# Model

In [5]:
def model_fn(hidden_dim=384, dropout=.5, pred_len=68, n_outputs=5):
    inputs_seq  = L.Input(shape=(None, 1), name='inputs_seq')  
    inputs_struct  = L.Input(shape=(None, 1), name='inputs_struct')  
    inputs_loop  = L.Input(shape=(None, 1), name='inputs_loop')
    inputs_bpps_max  = L.Input(shape=(None, 1), name='inputs_bpps_max')
    inputs_bpps_sum  = L.Input(shape=(None, 1), name='inputs_bpps_sum')
    inputs_bpps_mean  = L.Input(shape=(None, 1), name='inputs_bpps_mean')

    def _one_hot(x, num_classes):
        return K.squeeze(K.one_hot(K.cast(x, 'uint8'), num_classes=num_classes), axis=2)

    ohe_seq = L.Lambda(_one_hot, arguments={'num_classes': len(token2int_seq)}, input_shape=(None, 1))(inputs_seq)
    ohe_struct = L.Lambda(_one_hot, arguments={'num_classes': len(token2int_struct)}, input_shape=(None, 1))(inputs_struct)
    ohe_loop = L.Lambda(_one_hot, arguments={'num_classes': len(token2int_loop)}, input_shape=(None, 1))(inputs_loop)
    
    # Conv block
    conv_seq = L.Conv1D(filters=64, 
                        kernel_size=5, 
                        strides=1, 
                        padding='same')(ohe_seq)
    conv_struct = L.Conv1D(filters=64, 
                           kernel_size=5, 
                           strides=1, 
                           padding='same')(ohe_struct)
    conv_loop = L.Conv1D(filters=64, 
                         kernel_size=5, 
                         strides=1, 
                         padding='same')(ohe_loop)
    conv_bpps_max = L.Conv1D(filters=64, 
                             kernel_size=5, 
                             strides=1, 
                             padding='same')(inputs_bpps_max)
    conv_bpps_sum = L.Conv1D(filters=64, 
                             kernel_size=5, 
                             strides=1, 
                             padding='same')(inputs_bpps_sum)
    conv_bpps_mean = L.Conv1D(filters=64, 
                             kernel_size=5, 
                             strides=1, 
                             padding='same')(inputs_bpps_mean)
    
    x_concat = L.concatenate([conv_seq, conv_struct, conv_loop, conv_bpps_max, conv_bpps_sum, conv_bpps_mean], axis=-1, name='conv_concatenate')

    # Recurrent block
    x = L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(x_concat)
    
    x_rec = L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(x)
    x = L.Add()([x_rec, x])
    
    x_rec = L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(x)
    x = L.Add()([x_rec, x])
    
    
    # Since we are only making predictions on the first part of each sequence, we have to truncate it
    x_truncated = x[:, :pred_len]
    
    outputs = L.Dense(n_outputs, activation='linear', name='outputs')(x_truncated)

    model = Model(inputs=[inputs_seq, inputs_struct, inputs_loop, inputs_bpps_max, inputs_bpps_sum, inputs_bpps_mean], outputs=outputs)

    opt = optimizers.Adam(learning_rate=config['LEARNING_RATE'])
    model.compile(optimizer=opt, loss=MCRMSE)

    return model

model = model_fn()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs_seq (InputLayer)         [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
inputs_struct (InputLayer)      [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
inputs_loop (InputLayer)        [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, None, 4)      0           inputs_seq[0][0]                 
_______________________________________________________________________________________

# Pre-process

In [6]:
# Add bpps as features
bpps_max = []
bpps_sum = []
bpps_mean = []
for row in train.itertuples():
    probability = np.load(f'{database_base_path}/bpps/{row.id}.npy')
    bpps_max.append(probability.max(-1).tolist())
    bpps_sum.append((1-probability.sum(-1)).tolist())
    bpps_mean.append((1-probability.mean(-1)).tolist())
train = train.assign(bpps_max=bpps_max, bpps_sum=bpps_sum, bpps_mean=bpps_mean)

bpps_max = []
bpps_sum = []
bpps_mean = []
for row in test.itertuples():
    probability = np.load(f'{database_base_path}/bpps/{row.id}.npy')
    bpps_max.append(probability.max(-1).tolist())
    bpps_sum.append((1-probability.sum(-1)).tolist())
    bpps_mean.append((1-probability.mean(-1)).tolist())
test = test.assign(bpps_max=bpps_max, bpps_sum=bpps_sum, bpps_mean=bpps_mean)


feature_cols = ['sequence', 'structure', 'predicted_loop_type', 'bpps_max', 'bpps_sum', 'bpps_mean']
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
encoder_list = [token2int_seq, token2int_struct, token2int_loop, None, None, None]

train_features = np.array([preprocess_inputs(train, encoder_list[idx], [col]) for idx, col in enumerate(feature_cols)]).transpose((1, 0, 2, 3))
train_labels = np.array(train[pred_cols].values.tolist()).transpose((0, 2, 1))

public_test = test.query("seq_length == 107").copy()
private_test = test.query("seq_length == 130").copy()

x_test_public = np.array([preprocess_inputs(public_test, encoder_list[idx], [col]) for idx, col in enumerate(feature_cols)]).transpose((1, 0, 2, 3))
x_test_private = np.array([preprocess_inputs(private_test, encoder_list[idx], [col]) for idx, col in enumerate(feature_cols)]).transpose((1, 0, 2, 3))

# To use as stratified col
train['signal_to_noise_int'] = train['signal_to_noise'].astype(int)

# Training

In [7]:
AUTO = tf.data.experimental.AUTOTUNE
skf = StratifiedKFold(n_splits=config['N_USED_FOLDS'], shuffle=True, random_state=SEED)
history_list = []

oof = train[['id', 'SN_filter', 'signal_to_noise']].copy()
oof_preds = np.zeros(train_labels.shape)
test_public_preds = np.zeros((x_test_public.shape[0], config['PB_SEQ_LEN'], len(pred_cols)))
test_private_preds = np.zeros((x_test_private.shape[0], config['PV_SEQ_LEN'], len(pred_cols)))

for fold,(train_idx, valid_idx) in enumerate(skf.split(train_labels, train['signal_to_noise_int'])):
    if fold >= config['N_USED_FOLDS']:
        break
    print(f'\nFOLD: {fold+1}')
    
    # Create clean and noisy datasets
    train_clean_idxs = np.intersect1d(train[train['signal_to_noise'] > 1].index, train_idx)
    valid_clean_idxs = np.intersect1d(train[train['signal_to_noise'] > 1].index, valid_idx)

    ### Create datasets
    x_train = train_features[train_clean_idxs]
    y_train = train_labels[train_clean_idxs]
    x_valid = train_features[valid_clean_idxs]
    y_valid = train_labels[valid_clean_idxs]
    
    train_ds = get_dataset(x_train, y_train, labeled=True, shuffled=True, batch_size=config['BATCH_SIZE'], buffer_size=AUTO, seed=SEED)
    valid_ds = get_dataset(x_valid, y_valid, labeled=True, shuffled=False, batch_size=config['BATCH_SIZE'], buffer_size=AUTO, seed=SEED)
    oof_ds = get_dataset(train_features[valid_idx], labeled=False, shuffled=False, batch_size=config['BATCH_SIZE'], buffer_size=AUTO, seed=SEED)
    test_public_ds = get_dataset(x_test_public, labeled=False, shuffled=False, batch_size=config['BATCH_SIZE'], buffer_size=AUTO, seed=SEED)
    test_private_ds = get_dataset(x_test_private, labeled=False, shuffled=False, batch_size=config['BATCH_SIZE'], buffer_size=AUTO, seed=SEED)


    ### Model
    K.clear_session()
    model = model_fn()

    model_path = f'model_{fold}.h5'
    es = EarlyStopping(monitor='val_loss', mode='min', patience=config['ES_PATIENCE'], restore_best_weights=True, verbose=1)
    rlrp = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=5, verbose=1)
    
    ### Train
    history = model.fit(train_ds,
                        validation_data=valid_ds,
                        callbacks=[es, rlrp], 
                        epochs=config['EPOCHS'], 
                        batch_size=config['BATCH_SIZE'],
                        verbose=2).history
      
    history_list.append(history)
    # Save last model weights
    model.save_weights(model_path)

    ### Inference
    oof_preds[valid_idx] = model.predict(oof_ds)
    
    # Short sequence (public test)
    model = model_fn(pred_len= config['PB_SEQ_LEN'])
    model.load_weights(model_path)
    test_public_preds += model.predict(test_public_ds) * (1 / config['N_USED_FOLDS'])
    
    # Long sequence (private test)
    model = model_fn(pred_len= config['PV_SEQ_LEN'])
    model.load_weights(model_path)
    test_private_preds += model.predict(test_private_ds) * (1 / config['N_USED_FOLDS'])


FOLD: 1
Epoch 1/120
27/27 - 5s - loss: 0.5897 - val_loss: 0.3930
Epoch 2/120
27/27 - 3s - loss: 0.3745 - val_loss: 0.3573
Epoch 3/120
27/27 - 3s - loss: 0.3515 - val_loss: 0.3435
Epoch 4/120
27/27 - 3s - loss: 0.3394 - val_loss: 0.3340
Epoch 5/120
27/27 - 3s - loss: 0.3295 - val_loss: 0.3270
Epoch 6/120
27/27 - 3s - loss: 0.3241 - val_loss: 0.3231
Epoch 7/120
27/27 - 3s - loss: 0.3186 - val_loss: 0.3211
Epoch 8/120
27/27 - 3s - loss: 0.3144 - val_loss: 0.3142
Epoch 9/120
27/27 - 3s - loss: 0.3124 - val_loss: 0.3115
Epoch 10/120
27/27 - 3s - loss: 0.3052 - val_loss: 0.3065
Epoch 11/120
27/27 - 3s - loss: 0.3003 - val_loss: 0.2990
Epoch 12/120
27/27 - 3s - loss: 0.2937 - val_loss: 0.2940
Epoch 13/120
27/27 - 3s - loss: 0.2892 - val_loss: 0.2884
Epoch 14/120
27/27 - 3s - loss: 0.2841 - val_loss: 0.2840
Epoch 15/120
27/27 - 3s - loss: 0.2794 - val_loss: 0.2783
Epoch 16/120
27/27 - 3s - loss: 0.2765 - val_loss: 0.2756
Epoch 17/120
27/27 - 3s - loss: 0.2746 - val_loss: 0.2689
Epoch 18/120
2

## Model loss graph


FOLD: 1
Train 0.18280 Validation 0.21822

FOLD: 2
Train 0.17713 Validation 0.21156

FOLD: 3
Train 0.18012 Validation 0.21220

FOLD: 4
Train 0.18918 Validation 0.21718

FOLD: 5
Train 0.15970 Validation 0.20982


In [8]:
for fold, history in enumerate(history_list):
    print(f'\nFOLD: {fold+1}')
    print(f"Train {np.array(history['loss']).min():.5f} Validation {np.array(history['val_loss']).min():.5f}")

plot_metrics_agg(history_list)

# Post-processing

In [9]:
# Assign values to OOF set
# Assign labels
for idx, col in enumerate(pred_cols):
    val = train_labels[:, :, idx]
    oof = oof.assign(**{col: list(val)})
    
oof.to_csv('oof.csv', index=False)

# Assign preds
for idx, col in enumerate(pred_cols):
    val = oof_preds[:, :, idx]
    oof = oof.assign(**{f'{col}_pred': list(val)})
    

# Assign values to test set
preds_ls = []

for df, preds in [(public_test, test_public_preds), (private_test, test_private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)

# Model evaluation

In [10]:
display(evaluate_model(train, train_labels, oof_preds, pred_cols))

,Metric/MCRMSE,Complete,Clean (SN),Noisy (SN),Clean (signal),Noisy (signal)
0,Overall,0.831599,0.221072,1.392340,0.235619,2.245042
1,reactivity,0.615293,0.205596,1.018593,0.210246,1.638311
2,deg_Mg_pH10,0.554449,0.254223,0.884931,0.254954,1.406691
3,deg_pH10,1.181050,0.235508,2.004795,0.259486,3.247761
4,deg_Mg_50C,0.747368,0.208204,1.252204,0.222825,2.016766
5,deg_50C,1.059837,0.201831,1.801178,0.230585,2.915683


# Visualize test predictions

In [11]:
submission = pd.read_csv(database_base_path + 'sample_submission.csv')
submission = submission[['id_seqpos']].merge(preds_df, on=['id_seqpos'])

# Test set predictions

In [12]:
display(submission.head(10))
display(submission.describe())

submission.to_csv('submission.csv', index=False)

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.838301,0.701454,1.885497,0.593468,0.783132
1,id_00073f8be_1,2.383499,3.363863,4.288415,3.311637,2.763976
2,id_00073f8be_2,1.604351,0.662435,0.655922,0.821367,0.683614
3,id_00073f8be_3,1.345127,1.221058,1.255626,1.743012,1.783763
4,id_00073f8be_4,0.876356,0.560314,0.567863,0.856327,0.906815
5,id_00073f8be_5,0.747541,0.711489,0.726952,0.782690,0.788758
6,id_00073f8be_6,0.447063,0.787629,0.855672,0.842167,0.795941
7,id_00073f8be_7,0.525763,0.852756,1.033413,0.826384,1.166770
8,id_00073f8be_8,0.166894,0.597182,0.614280,0.636773,0.523701
9,id_00073f8be_9,0.044288,0.206390,0.197758,0.227062,0.260883


,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
count,457953.000000,457953.000000,457953.000000,457953.000000,457953.000000
mean,0.330470,0.412208,0.403139,0.346153,0.368252
std,0.361390,0.398544,0.433946,0.383857,0.339789
min,-0.178093,-1.214927,-3.031251,-2.438456,-2.404608
25%,0.080720,0.183262,0.151341,0.099730,0.124544
50%,0.207049,0.301090,0.296969,0.251615,0.299193
75%,0.475574,0.496885,0.523879,0.486338,0.539246
max,3.687229,5.058799,5.936711,4.225202,3.612676
